In [1]:
#!pip install -U kaleido

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans
from sklearn.ensemble import BaggingRegressor

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer, gen_rbf_X, create_clus_split
from spe.relaxed_lasso import RelaxedLasso, BaggedRelaxedLasso

In [3]:
err_cmp = ErrorComparer()

In [151]:
niter = 100

n=10**2
p=200
s=30

k=10
max_depth=6
n_estimators=100

In [152]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()

In [153]:
rho = 0.5

D = np.abs(c_x - c_x[:,None]) + np.abs(c_y - c_y[:,None])
Sigma_t = rho**D

# m = 20
# c = np.zeros(n)
# c[:m] = [rho**i for i in np.arange(m)]
# Sigma_t = toeplitz(c)

# b = n / m
# c = [rho**i for i in np.arange(m)] 
# Sigma_t = block_diag(*[toeplitz(c) for _ in np.arange(b)])

Chol_t = np.linalg.cholesky(Sigma_t)
# Sigma_t = np.eye(n)
# Chol_t = np.eye(n)

In [154]:
# c_x = np.random.uniform(0,10,size=n)
# c_y = np.random.uniform(0,10,size=n)

# rn = int(np.sqrt(n))
# ctr_x = np.random.uniform(0,10,size=rn)
# ctr_y = np.random.uniform(0,10,size=rn)
# c_x = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_x])
# c_y = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_y])

X = gen_rbf_X(c_x, c_y, p)
idx = create_clus_split(int(np.sqrt(n)), int(np.sqrt(n)))
X_sample = X[idx,:]
Chol_t_sample = Chol_t[idx,:][:,idx]
c_x_sample = c_x[idx]
c_y_sample = c_y[idx]

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [155]:
# xv, yv = np.meshgrid(np.arange(nx), np.arange(ny))
# pts = np.stack([xv.ravel(), yv.ravel()]).T
# n = nx*ny
# rn = int(np.sqrt(n))
# ctr = np.random.choice(pts.shape[0], size=rn, replace=True)
# ctr = pts[ctr]z
# tr_idx = np.vstack([[pt + np.array((1.25*np.random.randn(2)).astype(int)) for _ in np.arange(rn)] for pt in ctr])
# tr_idx = np.maximum(0, tr_idx)
# tr_idx[:,0] = cx = np.minimum(nx-1, tr_idx[:,0])
# tr_idx[:,1] = cy = np.minimum(ny-1, tr_idx[:,1])
# tr_idx = np.unique(np.ravel_multi_index(tr_idx.T, (nx,ny)))
# tr_idx.shape

In [156]:
(test_err,
 kfcv_err,
 spcv_err,
 brel_err) = err_cmp.compareForestTrTsFair(n=n,
                                       p=p,
                                       X=X_sample,
                                       beta=beta,
                                       coord=np.stack([c_x_sample, c_y_sample]).T,
                                       Chol_t=Chol_t_sample,
                                       niter=niter,
                                       max_depth=max_depth,
                                       n_estimators=n_estimators,
                                       k=k,
                                       bootstrap_type='blur')
    

0
0.7692307692307693
10
20
30
40
50
60
70
80
90


In [157]:
#%debug

In [158]:
# groups = KMeans(n_clusters=10).fit(tr_idx).labels_

In [159]:
# px.scatter(pd.DataFrame({'x': tr_idx[:,0], 'y': tr_idx[:,1], 'g': groups}), x='x', y='y', color='g')

In [160]:
risk = test_err.mean()
risk_kfcv = kfcv_err.mean()
risk_spcv = spcv_err.mean()
# risk_rela = rela_err.mean()
risk_brel = brel_err.mean()
risk, risk_kfcv, risk_spcv, risk_brel

(0.9646174863071987,
 0.5256784878084707,
 0.6796774537941128,
 0.9617271806646016)

In [161]:
df = pd.DataFrame({'GenCp': (brel_err.T),# - test_err.T) / test_err.T,
                   'KFCV': (kfcv_err.T),# - test_err.T) / test_err.T,
                   'SPCV': (spcv_err.T)})# - test_err.T) / test_err.T})

In [166]:
(pd.DataFrame({'GenCp': (brel_err.T),
               'KFCV': (kfcv_err.T),
               'SPCV': (spcv_err.T)})/risk).to_csv('~/Downloads/rff_df.csv')

In [162]:
fig_rela = px.box((df - risk) / risk, 
             labels={
                     "variable": "Method",
                     "value": "Relative MSE"
                     },
             title="lin correction",
             points=False)
fig_rela.update_traces(boxmean=True)

fig_rela.add_hline(y=0., line_color='red')

In [163]:
fig = go.Figure(data=go.Scatter(
        x=[0, 1, 2],
        y=((df - risk)/risk).mean(),
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=((df - risk)/risk).std(),
            visible=True)
    ))
fig.show()

In [164]:
pd.DataFrame((df/risk).mean())

,0
GenCp,0.997004
KFCV,0.544961
SPCV,0.704608


In [168]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75),
        arrayminus=(df/risk).quantile(.25))
))
fig.add_hline(y=1., line_color='red')
fig.update_layout(
    title="RF Test Error Estimates",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
# ticktext= []
# for i in np.arange()
fig.update_layout(
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0.,
        dtick = 0.25,
#         ticktext = ticktext,
    )
)
fig.show()